# Cutoff Power-Law or Comptonized Power-Law

This program shows you how to calcualte the integrated photon and energy fluxes of the COPL using two different methods.

1.  Integrating the Function in Wolfram Alpha and then passing an energy array to the integrated function.
2.  Using scipy.integrate to numerically integrate the function. This way is more precise.

### The equation for the Cutoff Power Law can be found here:
https://heasarc.gsfc.nasa.gov/xanadu/xspec/manual/XSmodelCutoffpl.html

#### \** NOTE:  I switch the sign on the cutoff power-law index.

Original Function, before integration: 
    
\begin{equation}
f\left(E\right) = N \times \left(\frac{E}{100}\right)^{\alpha} \times \exp \left( -\frac{E}{E_{C}} \right)  dE \ \ \ \ \ \ \ \         
\end{equation}

where $\alpha$ is the cutoff power-law index or low-energy slope.

$E_C$ is the high energy cutoff.

### My version of this equation:
I use different parameters to represent the variables that can be used within python to run calculations.

Original Function, before integration: 
    
    cplIndex or alpha: low energy index
    cutoff:  high-energy cutoff
    N:     normalization
    E:     energy to integrate over, 10 to 10,000 keV


    f(E) = N * (E**(cplIdx) * exp(-E/cutoff)) dE

### Converstion between High Energy Cutoff and Epeak energy:
Sometimes you'll see the energy as Epeak.

    epk    = 580.22058
    alpha  = -1.0534484

    ecut   = (epk)/(alpha + 2.); ecut
    ecut   = (580.22058)/(-1.0534484 + 2.)
    # ecut = 612.9835711016706

\begin{equation}
E_{pk} = E_{cutoff} \times \left( \alpha + 2.0 \right) \ \ \ \ \ 
\end{equation}

# Begin Program

In [1]:
from __future__ import division
import numpy as np
from scipy import integrate

### Constants

In [2]:
keVtoerg    = 1.60217657E-9
emin        = 10.0
emax        = 10000.0

### XSPEC Parameter Names
    pars = [cplIndex, highEcut, norm]

In [3]:
pars     = [-1.0534484, 612.9835711016706, 0.017022533]  

# normalization
norm   = pars[-1]

def get_parVals():
    pars     = [-1.0534484, 612.9835711016706, 0.017022533]  
    return pars


# FUNCTION INTEGRATED

Make a flux array of 5000 elements that will be filled with flux increments.
An energy array with 5000 energies ranging from 10 to 10,000 keV in log space will be passed to the integrated COPL function and will calculate an increment of flux for each energy element passed to it.  
Those increments will be stored in the flux array of 5000 elements.

After running copl(engs, flux, *pars) the fluxes (un-normalized) will be appended to the flux array.  Then you can multiply each flux element by the normalization factor.  
You could pass the normalization to the copl function and multiply it within the function instead of at the end.  
I do not do that here because the equation (shown below) was the exact one  used during PYXSPEC fitting.  The only difference between this function and the PYXSPEC one is that the order the variables passed to the copl function in PYXSPEC are:  copl(engs, params, flux)


If you sum up the 5000 increments in the flux array and then multiply by the the normalization, you are getting the integrated flux from 10 to 10,000 keV.
You are essentially summing up the flux area under the flux curve.


In this example, we leave the COPL normalization parameter out of the function and multiply it at the end.  We do this because PYXSPEC uses this exact function for copl (as below) during the Maximum Likelihood fitting process of parameter estimation.  

This is how PYXSPEC estiamtes its flux calculations when you use the commands:

    AllModels.setEnergies("10. 10000.")
    AllModels.calcFlux("10. 10000.0 err")
 
This is a crude method to estimate the flux and is not the  most accurate way.  The best way to calculate the integrated flux is to take the original function and numerically integrate it with scipy.integrate.quad for quadrature integration.

    Photon Flux Units:  photons s^-1 cm^-2
    Energy Flux Units:  ergs s^-1 cm^-2


## Wolfram Alpha Integration of the Cutoff Power-Law:

http://www.wolframalpha.com/input/?i=(x%5E(a)+*+exp(-x%2FC)+)+dx


When using Wolfram Alpha to integrate, be careful which letters you use for parameters.  Wolfram alpha has some letters set aside to mean something.  If they are used, you will not get the right answer. For example, E stands for exponential. Do NOT use E for energy.

N can be left out of integration. Simply multiply it back on at the end. The more parameters you have, the less likely Wolfram Alpha will calculate the function without a calculation time issue.

    a - cutoff power-law index
    C - high energy cutoff
    N - normalization
    x - energy


In [4]:
'''
fp.gammainc( cplIdx + 1., a = (engs[i]/cutoff) )
a - the lower integral limit.
b - the upper integral limit, default: b is infinity.
'''

def copl(engs, flux, *params):
    '''
    I do not use a here for the cutoff power-law index (or alpha) because of the 
    a used in the lower limit of the gammainc calculation.  I didn't want them 
    to become confused.
    
    '''
    from mpmath import gammainc
    from mpmath import fp
    import time
    
    start_time = time.time()
    
    for i in range(len(engs)-1):
        cplIdx   = float(params[0])    # low-energy slope (alpha)
        cutoff   = float(params[1])    # cutoff energy
        multiplier    = (100.0**(-cplIdx))*(cutoff**(cplIdx+1.))
        lowIntegral   = float(fp.gammainc( cplIdx + 1., a=(engs[i]/cutoff)) ) 
        highIntegral  = float(fp.gammainc( cplIdx + 1., a=(engs[i+1]/cutoff)) )
        val           = multiplier * (lowIntegral - highIntegral)
        flux[i]       = val
    stop_time = time.time() - start_time 
    print('time: %f seconds'%(stop_time))

#### We ran this function twice:
Once with fp.gammainc and once without it, gammainc.

fp stands for fast low-precision arithmetic.  This speeds up the process.

    With fp:     2.908276 seconds  
        Fluxes: 6.167920487 and 1.535317095e-06
        
    Without fp:  11.440344 seconds
        Fluxes: 6.167920487 and 1.535317095e-06
        
You can see that the lower precision doesn't change the flux values.
http://docs.sympy.org/0.6.7/modules/mpmath/basics.html

In [5]:
N      = 5000
engs   = np.logspace(1, 4, N)
flux   = np.zeros(N)

# WILL STORE CALCULATIONS IN FLUX ARRAY.
copl(engs, flux, *pars)
flux_ph = np.sum(flux) * norm

# NO NEED TO MAKE AN ESBPL FUNCTION.  MULTIPLY ENGS BY THE FLUX.
flux_en = np.sum(flux * engs * keVtoerg) * norm

print(
'''
Photon Flux:  %.9f \t photons s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(flux_ph, flux_en))

time: 3.512194 seconds

Photon Flux:  6.167920487 	 photons s^-1 cm^-2
Energy Flux:  1.535317095e-06 	 ergs s^-1 cm^-2



## FUNCTION WITHOUT BEING INTEGRATED.

In [6]:
def copl(energy):
    from numpy import exp
    cplIndex, highEcut, norm = get_parVals()
    a = float(cplIndex)
    C = float(highEcut)
    N = float(norm)
    eng = energy
    return N * ((eng/100.0)**(a)) * (exp(-eng/C))

def ecopl(energy):
    eng = energy
    return eng * copl(eng)

In [7]:
Flux_Ph = integrate.quad(copl, emin, emax, limit=100)[0]
Flux_En = integrate.quad(ecopl, emin, emax, limit=100)[0] * keVtoerg

print(
'''
Photon Flux:  %.9f \t photons s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(Flux_Ph, Flux_En))


Photon Flux:  6.167920487 	 photons s^-1 cm^-2
Energy Flux:  1.536378106e-06 	 ergs s^-1 cm^-2

